In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import requests
import time
import io
import json
import re

In [14]:
def auth_headers(token):
    return {'Authorization': 'OAuth {token}'.format(token=token)}

### Выгрузка из Аппметрики

In [3]:
responses = []
token = "YOUR_TOKEN"
apikey = "YOUR_APIKEY"

In [4]:
date_since = '2019-05-23'
date_until = '2019-05-23'
columns_to_load = ['event_name', 'event_timestamp', 'appmetrica_device_id', 'os_name']
columns_to_extract = ['event_name', 'event_timestamp', 'appmetrica_device_id', 'os_name']
columns_to_load = str(columns_to_load).replace("', '", "%2C").replace("['", '').replace("']", '')
operating_systems = ['ios']

в app_details_url лежит сам запрос, поля указываются в конце запроса через запятую (%2C)

In [8]:
def appmetrica_loader(apikey, date_since, date_until, token):
    app_details_url = """https://api.appmetrica.yandex.ru/logs/v1/export/events.csv?application_id={apikey}&date_since={date_since}%2000%3A00%3A00&date_until={date_until}%2023%3A59%3A59&date_dimension=default&use_utf8_bom=true&fields={columns_to_load}""".format(apikey=apikey
                                                                                 , date_until=date_until
                                                                                 , date_since=date_since
                                                                                 , columns_to_load=columns_to_load)
    
    while True:
        r = requests.get(app_details_url, headers=auth_headers(token))
        if r.status_code == 200:
            s=requests.get(app_details_url, headers=auth_headers(token)).content
            df=pd.read_csv(io.StringIO(s.decode('utf-8')))
            return(df)
            break;
        if r.status_code != 202:
            print('Bad response code')
            exit()
        print('Result is not ready yet. Waiting')
       

In [9]:
def filter_and_convert(df, my_id):
    mydf = df[df['appmetrica_device_id'] == my_id].sort_values(by='event_timestamp')  
    mydf['event_timestamp'] = pd.to_datetime(mydf['event_timestamp'] * (10 ** 9))  # convert timestamps, would be better to use _convert_timestamps from utils
    return mydf

def to_seconds(dt):
    return int((dt - pd.datetime(1970,1,1)).total_seconds())


каждый раз надо указывать новый временной промежуток, потому что на повторный запрос аппметрика будет отправлять предыдущий ответ

In [10]:
df = appmetrica_loader(apikey, date_since, date_until, token)

Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is 

Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting
Result is not ready yet. Waiting


In [11]:
df

,event_name,event_timestamp,appmetrica_device_id
0,RWbCarriageBack,1558563665,16868851947379889182
1,RWbCarriageSelection,1558563650,16868851947379889182
2,RWbCarriageSelection,1558561867,1143762740515707298
3,RWbChoosingOnSchemeBack,1558563663,16868851947379889182
4,RWbMainScreenSearch,1558561567,10063266143056392000
5,RWbMainScreenSearch,1558561590,10063266143056392000
6,RWbMainScreenSearch,1558561619,10063266143056392000
7,RWbMainScreenSearch,1558561639,10063266143056392000
8,RWbMainScreenSearch,1558561642,10063266143056392000
9,RWbMainScreenSearch,1558561654,10063266143056392000


### Выгрузка из Метрики

In [1]:
def auth_headers(token):
    return {'Authorization': 'OAuth {token}'.format(token=token)}

In [1]:
id_app = "YOUR_COUNTER" #номер счетчика
token_m = "YOUR_TOKEN" #отладочный токен

#### Для аггрегированных отчетов

In [25]:
import requests
import sys

header = {'Authorization': 'OAuth {token}'.format(token=token_m)}
ids = {
    'Counter': id_app,
}

payload = {
    'metrics': 'ym:s:visits,ym:s:users', #важно сохранить такой формат записи
    'date1': '2019-07-16',
    'date2': '2019-07-18',
    #'filters': "ym:s:deviceCategory!='mobile'",
    'ids': id_app,
    'accuracy': 'full',
    'group': 'day',
    'expand': True,
    'pretty': True,
}

i = 1
for key, value in ids.items():
    payload['ids'] = value
    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data/drilldown', params=payload, headers=header)
    data = str(r.json()).split(",")
    i += 1
    payload['ids'] = value
    print('total', key, data)

total Counter ["{'query': {'ids': [53991979]", " 'dimensions': []", " 'metrics': ['ym:s:visits'", " 'ym:s:users']", " 'sort': ['-ym:s:visits']", " 'date1': '2019-07-16'", " 'date2': '2019-07-18'", " 'limit': 100", " 'offset': 1", " 'group': 'day'", " 'auto_group_size': '1'", " 'quantile': '50'", " 'offline_window': '21'", " 'attribution': 'Last'", " 'currency': 'RUB'", " 'adfox_event_id': '0'}", " 'data': [{'dimension': {}", " 'metrics': [148.0", ' 121.0]', " 'expand': False}]", " 'total_rows': 1", " 'total_rows_rounded': False", " 'sampled': False", " 'sample_share': 1.0", " 'sample_size': 148", " 'sample_space': 148", " 'data_lag': 97", " 'totals': [148.0", ' 121.0]', " 'min': [148.0", ' 121.0]', " 'max': [148.0", ' 121.0]}']


#### Для сырых логов

###### Названия нужных полей можно взять отсюда:

https://yandex.ru/dev/metrika/doc/api2/logs/fields/hits-docpage/


https://yandex.ru/dev/metrika/doc/api2/logs/fields/visits-docpage/

In [78]:
import requests
import sys

header = {'Authorization': 'OAuth {token}'.format(token=token_m)}
ids = {
    'Counter': id_app,
}
#Параметры
payload = {
    'fields': 'ym:pv:dateTime,ym:pv:watchID',
    'date1': '2019-07-16',
    'date2': '2019-07-17',
    'source': 'hits' # another option is 'visits'
}


total Counter ["{'requests': [{'request_id': 4088020", " 'counter_id': 53991979", " 'source': 'hits'", " 'date1': '2019-07-16'", " 'date2': '2019-07-18'", " 'fields': ['ym:pv:dateTime'", " 'ym:pv:watchID']", " 'status': 'processed'", " 'size': 16760", " 'parts': [{'part_number': 0", " 'size': 16760}]}", " {'request_id': 4089550", " 'counter_id': 53991979", " 'source': 'hits'", " 'date1': '2019-07-16'", " 'date2': '2019-07-18'", " 'fields': ['ym:pv:dateTime'", " 'ym:pv:watchID']", " 'status': 'processed'", " 'size': 16760", " 'parts': [{'part_number': 0", " 'size': 16760}]}", " {'request_id': 4089592", " 'counter_id': 53991979", " 'source': 'hits'", " 'date1': '2019-07-16'", " 'date2': '2019-07-18'", " 'fields': ['ym:pv:dateTime'", " 'ym:pv:watchID']", " 'status': 'processed'", " 'size': 16760", " 'parts': [{'part_number': 0", " 'size': 16760}]}", " {'request_id': 4090102", " 'counter_id': 53991979", " 'source': 'hits'", " 'date1': '2019-07-16'", " 'date2': '2019-07-18'", " 'fields': ['

In [110]:
from time import sleep

#Функция для отправки запроса и выгрузки
def get_results(payload, id_app):
    print('It may take a while')
    p = requests.post('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequests'.format(id_app=id_app), params=payload, headers=header).content
    request_id = json.load(io.StringIO(p.decode('utf-8')))['log_request']['request_id']
    idx = 1
    while True:
        print(f"Query: {idx}")
        r = requests.get('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequest/{request_id}/part/0/download'.format(id_app=id_app, request_id=request_id), headers=header).content
        if pd.read_csv(io.StringIO(r.decode('utf-8')), sep='\t').shape[0] == 0:
            idx += 1
            sleep(60)
        else:
            break
    data = pd.read_csv(io.StringIO(r.decode('utf-8')), sep='\t')
    return data


In [2]:
#Выгрузка
get_results(payload, id_app)

NameError: name 'get_results' is not defined

Запасной вариант, вручную то же самое

In [85]:
#Отправляем запрос на создание выгрузки, нам нужно забрать request_id, его подставляем в следующий запрос
import json
p = requests.post('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequests'.format(id_app=id_app), params=payload, headers=header).content
request_id = json.load(io.StringIO(p.decode('utf-8')))['log_request']['request_id']

In [104]:
r = requests.get('https://api-metrika.yandex.net/management/v1/counter/{id_app}/logrequest/{request_id}/part/0/download'.format(id_app=id_app, request_id=request_id), headers=header).content

In [ ]:
data = pd.read_csv(io.StringIO(r.decode('utf-8')), sep='\t')